# Hands on Object Relational Mapping in PyCram

This tutorial will walk you through the serialization of a minimal plan in pycram.
First we will import sqlalchemy, create an in memory database and connect a session to it.

In [2]:
import sqlalchemy
import sqlalchemy.orm

engine = sqlalchemy.create_engine("sqlite+pysqlite:///:memory:", echo=False)
session = sqlalchemy.orm.Session(bind=engine)
session

Next we create the database schema using the sqlalchemy functionality. For that we need to import the base class of pycram.orm.

In [3]:
import pycram.orm.base
import pycram.orm.action_designator
pycram.orm.base.Base.metadata.create_all(engine)
session.commit()

Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Failed to import Giskard messages, the real robot will not be available
Could not import RoboKudo messages, RoboKudo interface could not be initialized
pybullet build time: Nov 28 2023 23:51:11


In [4]:
pycram.orm.base.Base.metadata.tables

FacadeDict({'ProcessMetaData': Table('ProcessMetaData', MetaData(), Column('created_at', DateTime(), table=<ProcessMetaData>, nullable=False, server_default=DefaultClause(<sqlalchemy.sql.functions.current_timestamp at 0x7f7949378640; current_timestamp>, for_update=False)), Column('created_by', String(length=255), table=<ProcessMetaData>, nullable=False, default=ScalarElementColumnDefault('malineni')), Column('description', String(length=255), table=<ProcessMetaData>, nullable=False), Column('pycram_version', String(length=255), table=<ProcessMetaData>, default=ScalarElementColumnDefault('3d49733b84ed8d2aecee1722b1ed7d25271219e1')), Column('id', Integer(), table=<ProcessMetaData>, primary_key=True, nullable=False), schema=None), 'Designator': Table('Designator', MetaData(), Column('dtype', String(length=255), table=<Designator>, nullable=False), Column('process_metadata_id', Integer(), ForeignKey('ProcessMetaData.id'), table=<Designator>, nullable=False), Column('id', Integer(), table=<

Next we will write a simple plan where the robot parks his arms and then moves somewhere. We will construct a TaskTree around it such that we can serialize it later. As usual, we first create a world and then define the plan. By doing so, we obtain the task tree.

In [ ]:
from pycram.designators.action_designator import *
from pycram.designators.location_designator import *
from pycram.process_module import simulated_robot
from pycram.datastructures.enums import Arms, ObjectType, WorldMode
from pycram.tasktree import with_tree
from pycram.worlds.bullet_world import Object, BulletWorld
from pycram.designators.object_designator import *
from pycram.datastructures.pose import Pose
import anytree

world = BulletWorld(mode=WorldMode.GUI)
pr2 = Object("pr2", ObjectType.ROBOT, "pr2.urdf")
kitchen = Object("kitchen", ObjectType.ENVIRONMENT, "kitchen.urdf")
milk = Object("milk", ObjectType.MILK, "milk.stl", pose=Pose([1.3, 1, 0.9]))
cereal = Object("cereal", ObjectType.BREAKFAST_CEREAL, "breakfast_cereal.stl", pose=Pose([1.3, 0.7, 0.95]))
milk_desig = ObjectDesignatorDescription(names=["milk"])
cereal_desig = ObjectDesignatorDescription(names=["cereal"])
robot_desig = ObjectDesignatorDescription(names=["pr2"]).resolve()
kitchen_desig = ObjectDesignatorDescription(names=["kitchen"])

@with_tree
def plan():
    with simulated_robot:
        ParkArmsActionPerformable(Arms.BOTH).perform()
        MoveTorsoAction([0.3]).resolve().perform()
        pickup_pose = CostmapLocation(target=cereal_desig.resolve(), reachable_for=robot_desig).resolve()
        pickup_arm = pickup_pose.reachable_arms[0]
        NavigateAction(target_locations=[pickup_pose.pose]).resolve().perform()
        PickUpAction(object_designator_description=cereal_desig, arms=[pickup_arm], grasps=["front"]).resolve().perform()
        ParkArmsAction([Arms.BOTH]).resolve().perform()

        place_island = SemanticCostmapLocation("kitchen_island_surface", kitchen_desig.resolve(),
                                           cereal_desig.resolve()).resolve()

        place_stand = CostmapLocation(place_island.pose, reachable_for=robot_desig, reachable_arm=pickup_arm).resolve()

        NavigateAction(target_locations=[place_stand.pose]).resolve().perform()

        PlaceAction(cereal_desig, target_locations=[place_island.pose], arms=[pickup_arm]).resolve().perform()

        ParkArmsActionPerformable(Arms.BOTH).perform()

plan()

# set description of what we are doing
pycram.orm.base.ProcessMetaData().description = "Tutorial for getting familiar with the ORM."
task_tree = pycram.tasktree.task_tree
print(anytree.RenderTree(task_tree))

Scalar element defined multiple times: limit
Scalar element defined multiple times: limit
Unknown tag "rgba_color" in /robot[@name='milk_object']/link[@name='milk_main']/visual[1]/material[@name='white']
Unknown tag "rgba_color" in /robot[@name='milk_object']/link[@name='milk_main']/visual[1]/material[@name='white']
Unknown tag "rgba_color" in /robot[@name='cereal_object']/link[@name='cereal_main']/visual[1]/material[@name='white']
Unknown tag "rgba_color" in /robot[@name='cereal_object']/link[@name='cereal_main']/visual[1]/material[@name='white']


[INFO] [1729682038.844590]: Ontology [http://www.ease-crc.org/ont/SOMA-HOME.owl#]'s name: SOMA-HOME has been loaded
[INFO] [1729682038.845325]: - main namespace: SOMA-HOME
[INFO] [1729682038.845948]: - loaded ontologies:
[INFO] [1729682038.846276]: http://www.ease-crc.org/ont/SOMA-HOME.owl#
[INFO] [1729682038.846559]: http://www.ease-crc.org/ont/DUL.owl#
[INFO] [1729682038.846843]: http://www.ease-crc.org/ont/SOMA.owl#
[INFO] [1729682039.828522]: Waiting for IK service: /pr2_left_arm_kinematics/get_ik


Next we serialize the task tree by recursively inserting from its root.

In [1]:
task_tree.root.insert(session)

NameError: name 'task_tree' is not defined

We can look at our experiment (Process)MetaData to get some context on the data we just created.

In [6]:
from sqlalchemy import select

print(*session.scalars(select(pycram.orm.base.ProcessMetaData)).all())

ProcessMetaData(id=1, created_at=datetime.datetime(2024, 8, 21, 8, 55, 46), created_by='malineni', description='Tutorial for getting familiar with the ORM.', pycram_version='3d49733b84ed8d2aecee1722b1ed7d25271219e1')


Lastly we can look at various table to see how the structures got logged.
For example, we can get all the navigate actions that occurred.

In [7]:
navigations = session.scalars(select(pycram.orm.action_designator.NavigateAction)).all()
print(*navigations, sep="\n")

NavigateAction(id=3, process_metadata_id=1, dtype='NavigateAction', robot_state_id=3, robot_state=RobotState(id=3, pose_to_init=False, torso_height=0.3, type=<ObjectType.ROBOT: 8>, pose_id=3, process_metadata_id=1), pose_to_init=False, pose_id=4)
NavigateAction(id=12, process_metadata_id=1, dtype='NavigateAction', robot_state_id=6, robot_state=RobotState(id=6, pose_to_init=False, torso_height=0.3, type=<ObjectType.ROBOT: 8>, pose_id=12, process_metadata_id=1), pose_to_init=False, pose_id=13)


Due to the inheritance mapped in the ORM package, we can also obtain all executed actions with just one query. 

In [8]:
actions = session.scalars(select(pycram.orm.action_designator.Action)).all()
print(*actions, sep="\n")

ParkArmsAction(id=1, process_metadata_id=1, dtype='ParkArmsAction', robot_state_id=1, robot_state=RobotState(id=1, pose_to_init=False, torso_height=0.0, type=<ObjectType.ROBOT: 8>, pose_id=1, process_metadata_id=1), arm=<Arms.BOTH: 3>)
MoveTorsoAction(id=2, process_metadata_id=1, dtype='MoveTorsoAction', robot_state_id=2, robot_state=RobotState(id=2, pose_to_init=False, torso_height=0.0, type=<ObjectType.ROBOT: 8>, pose_id=2, process_metadata_id=1), position=0.3)
NavigateAction(id=3, process_metadata_id=1, dtype='NavigateAction', robot_state_id=3, robot_state=RobotState(id=3, pose_to_init=False, torso_height=0.3, type=<ObjectType.ROBOT: 8>, pose_id=3, process_metadata_id=1), pose_to_init=False, pose_id=4)
PickUpAction(id=5, process_metadata_id=1, dtype='PickUpAction', robot_state_id=4, robot_state=RobotState(id=4, pose_to_init=False, torso_height=0.3, type=<ObjectType.ROBOT: 8>, pose_id=6, process_metadata_id=1), object_to_init=False, arm='left', grasp='front', object_id=1)
ParkArmsAct

Of course all relational algebra operators, such as filtering and joining also work in pycram.orm queries. Let's say we need all the poses of objects, that were picked up by a robot. Since we defined a relationship between the PickUpAction table and the Object table and between the Object table and the Pose table in the ORM class schema, we can just use the join operator without any further specification:

In [9]:
# session.scalars(select(pycram.orm.action_designator.PickUpAction)).all()
taction = session.query(pycram.orm.base.RobotState).first()
taction.pose

Pose(id=1, orientation_to_init=False, position_to_init=False, time=datetime.datetime(2024, 8, 21, 8, 55, 11, 916287), frame='map', position_id=1, orientation_id=1, process_metadata_id=1)

In [10]:
session.query(pycram.orm.action_designator.ParkArmsAction).all()

[ParkArmsAction(id=1, process_metadata_id=1, dtype='ParkArmsAction', robot_state_id=1, robot_state=RobotState(id=1, pose_to_init=False, torso_height=0.0, type=<ObjectType.ROBOT: 8>, pose_id=1, process_metadata_id=1), arm=<Arms.BOTH: 3>),
 ParkArmsAction(id=11, process_metadata_id=1, dtype='ParkArmsAction', robot_state_id=5, robot_state=RobotState(id=5, pose_to_init=False, torso_height=0.3, type=<ObjectType.ROBOT: 8>, pose_id=11, process_metadata_id=1), arm=<Arms.BOTH: 3>),
 ParkArmsAction(id=18, process_metadata_id=1, dtype='ParkArmsAction', robot_state_id=8, robot_state=RobotState(id=8, pose_to_init=False, torso_height=0.3, type=<ObjectType.ROBOT: 8>, pose_id=20, process_metadata_id=1), arm=<Arms.BOTH: 3>)]

In [11]:
object_actions = (session.scalars(select(pycram.orm.base.Pose)
                  .join(pycram.orm.action_designator.PickUpAction.object)
                  .join(pycram.orm.object_designator.Object.pose))
                  .all())
print(*object_actions, sep="\n")


Pose(id=7, orientation_to_init=False, position_to_init=False, time=datetime.datetime(2024, 8, 21, 8, 55, 26, 909066), frame='map', position_id=7, orientation_id=7, process_metadata_id=1)


Did you notice, that for the joins we did not join the tables together in a typical sql kind of way, but rather used the relationships defined in the ORM classes and wrote joins like PickUpAction.object or Object.pose? This is because the ORM package automatically creates the joins for us, so we only have to join on the attributes that hold the relationship. This is a huge advantage over writing sql queries by hand, since we do not have to worry about the join conditions. 
This is a strong tool, but it is crucial to use it properly. Very important to note: The order of the joins matters! For instance, if we joined the Pose table with the Object table first, and placed the join between the PickUpAction table and the Object table second, sqlalchemy would have selected the Pose not from the join between all three tables, but rather from a join between the Pose and the Object table + from a join between the PickUpAction table and the Object table. These mistakes can lead to wrong results or even to errors (the above-mentioned example would actually lead to an error due to the Object table being accessed twice in two separate joins in the same query and therefore the column names of the Object tables would have been ambiguous and could not be used by sqlalchemy to join).

Make sure to check out the other examples of ORM querying.

If we want to filter for all successful tasks we can just add the filter operator:

In [12]:
successful_tasks = session.scalars(select(pycram.orm.tasktree.TaskTreeNode).where(pycram.orm.tasktree.TaskTreeNode.status == "SUCCEEDED"))
print(*successful_tasks, sep="\n")

TaskTreeNode(id=2, action_id=None, action=None, start_time=datetime.datetime(2024, 8, 21, 10, 55, 19, 109011), end_time=datetime.datetime(2024, 8, 21, 10, 55, 28, 431303), status=<TaskStatus.SUCCEEDED: 2>, reason=None, parent_id=1, parent=TaskTreeNode(id=1, action_id=None, action=None, start_time=datetime.datetime(2024, 8, 21, 10, 55, 3, 531084), end_time=None, status=<TaskStatus.RUNNING: 1>, reason=None, parent_id=None, parent=None, process_metadata_id=1), process_metadata_id=1)
TaskTreeNode(id=3, action_id=1, action=ParkArmsAction(id=1, process_metadata_id=1, dtype='ParkArmsAction', robot_state_id=1, robot_state=RobotState(id=1, pose_to_init=False, torso_height=0.0, type=<ObjectType.ROBOT: 8>, pose_id=1, process_metadata_id=1), arm=<Arms.BOTH: 3>), start_time=datetime.datetime(2024, 8, 21, 10, 55, 19, 109043), end_time=datetime.datetime(2024, 8, 21, 10, 55, 19, 613804), status=<TaskStatus.SUCCEEDED: 2>, reason=None, parent_id=2, parent=TaskTreeNode(id=2, action_id=None, action=None, 

As expected all but the root node succeeded, since the root node is still running.

Writing an extension to the ORM package is also done with ease. We need to create a new ActionDesignator class and its ORM equivalent, where we define our new table. Let's say we want to log all the things the robot says. We will create a new ActionDesignator class called Saying and its ORM equivalent called ORMSaying. 

In [13]:
from sqlalchemy.orm import Mapped, mapped_column, Session
from pycram.orm.action_designator import Action
from dataclasses import dataclass


# define ORM class from pattern in every pycram.orm class
class ORMSaying(Action):

    id: Mapped[int] = mapped_column(sqlalchemy.ForeignKey(f'{Action.__tablename__}.id'), primary_key=True, init=False)
    # since we do not want to add any custom specifications to our column, we don't even need to define mapped_column, sqlalchemy does this internally.
    text: Mapped[str] 

# define brand new action designator

@dataclass 
class SayingActionPerformable(ActionDesignatorDescription.Action):
    
    text: str
        
    @with_tree
    def perform(self) -> None:
        print(self.text)

    def to_sql(self) -> ORMSaying:
        return ORMSaying(self.text)

    def insert(self, session: Session, *args, **kwargs) -> ORMSaying:
        action = super().insert(session)
        session.add(action)
        session.commit()
        return action


Now we got our new ActionDesignator called Saying and its ORM version. Since this class got created after all other classes got inserted into the database (in the beginning of the notebook) we have to insert it manually. 

In [14]:
ORMSaying.metadata.create_all(bind=engine)

Now we can create and insert a Saying action. Since this is the last part where we interact with the BulletWorld, we can also close it.

In [15]:
# create a saying action and insert it
SayingActionPerformable("Patchie, Patchie; Where is my Patchie?").perform()
pycram.tasktree.task_tree.root.insert(session)
session.commit()

world.exit()

Patchie, Patchie; Where is my Patchie?


Inserting TaskTree into database: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 1247.84it/s]


It is notable that committing the object to the session fills its primary key. Hence, there is no worries about assigning unique IDs manually.
Finally, we can double-check that our object exists in the database.

In [16]:
session.scalars(select(ORMSaying)).all()

[ORMSaying(id=37, process_metadata_id=1, dtype='ORMSaying', robot_state_id=17, robot_state=RobotState(id=17, pose_to_init=False, torso_height=0.3, type=<ObjectType.ROBOT: 8>, pose_id=41, process_metadata_id=1), text='Patchie, Patchie; Where is my Patchie?')]